# Proyek Analisis Data: Air-quality-dataset
- **Nama:** Izzuddin Ahmad Afif
- **Email:** izzuddinafif@gmail.com
- **ID Dicoding:** izzuddinafif

## Menentukan Pertanyaan Bisnis

- Bagaimana tren konsentrasi PM2.5 di masing-masing stasiun dari waktu ke waktu?
- Apakah terdapat perbedaan signifikan dalam kualitas udara antar stasiun, dan faktor apa yang mungkin menyebabkannya?

## Import Semua Packages/Library yang Digunakan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sb
import streamlit as st
import os
import glob

## Data Wrangling

### Gathering Data

In [ ]:
# Load semua nama csv file dalam folder csv
folder_path = 'csv'
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Dictionary untuk menampung data setiap stasiun dengan key berupa nama stasiun
station_dataframes = {}

# Memuat semua data untuk setiap stasiun ke dalam dataframe
for file in csv_files:
    base_name = os.path.basename(file)
        
    parts = base_name.split('_')
    if len(parts) > 3:
        station = parts[2] # nama stasiun
        
        df = pd.read_csv(file)
        station_dataframes[station] = df

# tes salah satu stasiun
station_dataframes['Aotizhongxin'].head()

**Insight:**
- Semua file CSV memiliki struktur yang seragam yang menunjukkan bahwa data dikumpulkan dengan format yang standar. Hal ini akan memudahkan langkah cleaning data dan analisis selanjutnya. 
- File CSV yang berbeda per stasiun yang kemudian dikelompokkan menggunakan dictionary memungkinkan analisis per stasiun secara terpisah maupun perbandingan antar stasiun. 
- Melihat kolom year, month, day, dan hour, kita bisa mengidentifikasi bahwa data merupakan data time series. Ini membuka peluang untuk analisis tren temporal (misalnya, tren harian, bulanan, atau tahunan).
- Struktur data yang sudah terpisah per stasiun memudahkan penerapan teknik analisis lanjutan seperti perbandingan kualitas udara antar stasiun, analisis geospasial, ataupun clustering berdasarkan pola-pola tertentu.

### Assessing Data

In [ ]:
def assess(df):
    results = {}
    
    # 1. Bentuk DataFrame
    results["shape"] = df.shape
    
    # 2. Missing Values
    results["missing_values"] = df.isnull().sum().to_dict()
    
    # 3. Nilai Invalid (cek apakah bisa dikonversi ke float)
    numeric_cols = ["PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "RAIN", "WSPM"]
    invalid_values = {}
    for col in numeric_cols:
        count = 0
        if col in df.columns:
            for val in df[col]:
                if pd.notnull(val):
                    try:
                        float(val)
                    except Exception:
                        count += 1
        invalid_values[col] = count
    results["invalid_values"] = invalid_values
    
    # 4. Baris duplikat
    results["duplicate_rows"] = int(df.duplicated().sum())
    
    # 5. Nilai tidak akurat (nilai negatif untuk kolom yang seharusnya non-negatif)
    non_negative_cols = ["PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "RAIN", "WSPM"]
    inaccurate_values = {}
    for col in non_negative_cols:
        count = 0
        if col in df.columns:
            for val in df[col]:
                try:
                    if float(val) < 0:
                        count += 1
                except Exception:
                    continue
        inaccurate_values[col] = count
    results["inaccurate_values"] = inaccurate_values
    
    # 6. Nilai tidak konsisten
    inconsistent_values = {}
    for col in df.columns:
        if df[col].dtype == 'O':
            inconsistent_values[col] = df[col].dropna().unique().tolist()
    results["inconsistent_values"] = inconsistent_values
    
    # 7. Deteksi Outlier dengan metode IQR 
    outliers = {}
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    for col in numeric_columns:
        if df[col].nunique() <= 1:
            outliers[col] = 0
            continue
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[col] = sum(1 for x in df[col] if x < lower_bound or x > upper_bound)
    results["outliers"] = outliers
    
    return results

assessment_results = {}
for station, df in station_dataframes.items():
    assessment_results[station] = assess(df)

# Ringkasan khusus untuk PM2.5
summary_data = []
for station, res in assessment_results.items():
    summary_data.append({
       "Station": station,
       "Rows": res["shape"][0],
       "Missing_PM2.5": res["missing_values"].get("PM2.5", 0),
       "Invalid_PM2.5": res["invalid_values"].get("PM2.5", 0),
       "Duplicate_Rows": res["duplicate_rows"],
       "Outliers_PM2.5": res["outliers"].get("PM2.5", 0)
    })

summary_df = pd.DataFrame(summary_data)
print("Ringkasan assessment untuk PM2.5 per stasiun:")
print(summary_df.to_string())

# Visualisasi Missing PM2.5 per stasiun
plt.figure(figsize=(10, 6))
plt.bar(summary_df["Station"], summary_df["Missing_PM2.5"], color="skyblue")
plt.xlabel("Stasiun")
plt.ylabel("Jumlah Missing PM2.5")
plt.title("Missing PM2.5 per Stasiun")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Visualisasi Outlier PM2.5 per stasiun
plt.figure(figsize=(10, 6))
plt.bar(summary_df["Station"], summary_df["Outliers_PM2.5"], color="salmon")
plt.xlabel("Stasiun")
plt.ylabel("Jumlah Outlier PM2.5")
plt.title("Outlier PM2.5 per Stasiun")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


**Insight:**
- Semua file memiliki jumlah baris dan kolom yang sama (35.064 baris dan 18 kolom), yang menunjukkan bahwa data dikumpulkan dalam periode dan struktur yang konsisten antar stasiun.
- Jumlah nilai hilang pada kolom, terutama untuk PM2.5, bervariasi antar stasiun. Misalnya, ada stasiun yang memiliki missing value PM2.5 yang tinggi (seperti Shunyi dengan 913 nilai hilang) dibandingkan dengan stasiun lain (misalnya Wanliu dengan 382 nilai hilang). Ini mengindikasikan adanya variasi dalam kualitas pengumpulan data atau masalah pencatatan di masing-masing lokasi.
- Tidak ditemukan baris duplikat dan invalid pada data.
- Deteksi outlier menggunakan metode IQR menunjukkan adanya jumlah pencilan yang cukup tinggi pada kolom PM2.5 (misalnya 1626 outlier di Huairou).

### Cleaning Data

In [ ]:
def impute_outliers(s, lower_quantile=0.25, upper_quantile=0.75, k=1.5):
    # Hitung Q1, Q3, dan IQR dari seri s
    Q1 = s.quantile(lower_quantile)
    Q3 = s.quantile(upper_quantile)
    IQR = Q3 - Q1
    lower_bound = Q1 - k * IQR
    upper_bound = Q3 + k * IQR
    median_val = s.median()
    
    # Buat salinan seri untuk diimputasi
    s_imputed = s.copy()
    s_imputed[s < lower_bound] = median_val
    s_imputed[s > upper_bound] = median_val
    return s_imputed

cleaned_dataframes = {}
stats_summary = []

for station, df in station_dataframes.items():
    # Menggabungkan kolom tanggal menjadi kolom datetime
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
    df.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
    # Memindahkan kolom datetime ke posisi kedua
    datetime_col = df.pop('datetime')
    df.insert(1, 'datetime', datetime_col)
    
    # Pastikan kolom PM2.5 berupa numerik
    df['PM2.5'] = pd.to_numeric(df['PM2.5'], errors='coerce')
    
    # Imputasi missing value untuk PM2.5 dengan interpolasi linier (berdasarkan urutan waktu)
    df.sort_values('datetime', inplace=True)
    df['PM2.5_clean'] = df['PM2.5'].interpolate(method='linear')
    
    # Imputasi outlier: ganti nilai outlier di PM2.5_clean dengan median
    df['PM2.5_imputed'] = impute_outliers(df['PM2.5_clean'])
    
    cleaned_dataframes[station] = df
    
    # Statistik ringkasan untuk PM2.5
    original_stats = df['PM2.5'].describe()
    clean_stats = df['PM2.5_clean'].describe()
    imputed_stats = df['PM2.5_imputed'].describe()
    
    stats_summary.append({
        "Station": station,
        "Original_Mean": original_stats.get('mean', np.nan),
        "Original_Median": original_stats.get('50%', np.nan),
        "Clean_Mean": clean_stats.get('mean', np.nan),
        "Clean_Median": clean_stats.get('50%', np.nan),
        "Imputed_Mean": imputed_stats.get('mean', np.nan),
        "Imputed_Median": imputed_stats.get('50%', np.nan)
    })

stats_df = pd.DataFrame(stats_summary)
print("Summary Statistik PM2.5 per stasiun:")
print(stats_df)

# Time Series dan Boxplot untuk stasiun Aotizhongxin sebagai contoh
station = "Aotizhongxin"
print("Keys di cleaned_dataframes:", list(cleaned_dataframes.keys()))
df_plot = cleaned_dataframes[station]

plt.figure(figsize=(12,6))
plt.plot(df_plot['datetime'], df_plot['PM2.5'], label='Original', alpha=0.5)
plt.plot(df_plot['datetime'], df_plot['PM2.5_clean'], label='Interpolated', alpha=0.8)
plt.plot(df_plot['datetime'], df_plot['PM2.5_imputed'], label='Outlier-Imputed', alpha=0.8)
plt.xlabel("Datetime")
plt.ylabel("PM2.5")
plt.title(f"Time Series PM2.5 - {station}")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,6))
plt.boxplot([df_plot['PM2.5'].dropna(), df_plot['PM2.5_imputed'].dropna()], labels=["Original", "Outlier-Imputed"])
plt.title(f"Boxplot PM2.5 - {station}")
plt.ylabel("PM2.5")
plt.tight_layout()
plt.show()


**Insight:**
- Dengan menggabungkan kolom tanggal (year, month, day, hour) menjadi satu kolom datetime dan mengurutkan data berdasarkan waktu, kita mendapatkan deret waktu yang lebih konsisten. Hal ini memudahkan analisis tren PM2.5 secara time series.
- Penggunaan interpolasi linier untuk mengisi missing data membuat data PM2.5 menjadi lebih kontinu. Jika missing data tidak diisi, bisa terjadi celah yang mengganggu analisis tren. Dengan imputasi, distribusi data menjadi lebih lengkap.
- Dari statistik awal, nilai rata-rata (mean) PM2.5 cenderung lebih tinggi karena adanya nilai ekstrim. Setelah dilakukan imputasi outlier dengan mengganti nilai ekstrim menggunakan median, nilai mean menurun secara signifikan, sedangkan median tetap stabil. Ini menunjukkan bahwa outlier sebelumnya memberikan bias pada perhitungan rata-rata.
- Setelah proses cleaning, data PM2.5 menjadi lebih homogen dan siap untuk analisis tren serta perbandingan kualitas udara antar stasiun. Data yang sudah dibersihkan memberikan dasar yang lebih solid untuk menjawab pertanyaan bisnis, misalnya apakah terdapat tren penurunan atau kenaikan konsentrasi PM2.5 dan perbedaan antar stasiun.



## Exploratory Data Analysis (EDA)

### Explore ...

In [ ]:

# 1. Time Series Plot untuk salah satu stasiun (contoh: Aotizhongxin)
if "Aotizhongxin" in cleaned_dataframes:
    df_aotizhongxin = cleaned_dataframes["Aotizhongxin"]
    plt.figure(figsize=(12,6))
    plt.plot(df_aotizhongxin['datetime'], df_aotizhongxin['PM2.5_imputed'], label="PM2.5 (Imputed)", color="blue")
    plt.xlabel("Datetime")
    plt.ylabel("PM2.5")
    plt.title("Time Series PM2.5 (Imputed) - Aotizhongxin")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("Stasiun Aotizhongxin tidak ditemukan.")

# 2. Agregasi Bulanan untuk stasiun Aotizhongxin
if "Aotizhongxin" in cleaned_dataframes:
    df_aotizhongxin['month'] = df_aotizhongxin['datetime'].dt.month
    monthly_avg = df_aotizhongxin.groupby('month')['PM2.5_imputed'].mean()
    plt.figure(figsize=(10,6))
    monthly_avg.plot(kind='bar', color='skyblue')
    plt.xlabel("Bulan")
    plt.ylabel("Rata-rata PM2.5 (Imputed)")
    plt.title("Rata-rata PM2.5 Bulanan - Aotizhongxin")
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

# 3. Perbandingan PM2.5 antar Stasiun (Boxplot)
all_data = []
for station, df in cleaned_dataframes.items():
    temp = df[['datetime', 'PM2.5_imputed']].copy()
    temp['Station'] = station
    all_data.append(temp)
all_data_df = pd.concat(all_data)
plt.figure(figsize=(12,6))
sns.boxplot(x="Station", y="PM2.5_imputed", data=all_data_df)
plt.xlabel("Stasiun")
plt.ylabel("PM2.5 (Imputed)")
plt.title("Perbandingan Distribusi PM2.5 antar Stasiun")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 4. Korelasi Antar Parameter untuk stasiun Aotizhongxin
if "Aotizhongxin" in cleaned_dataframes:
    numeric_cols = ["PM2.5_imputed", "PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "RAIN", "WSPM"]
    corr_matrix = df_aotizhongxin[numeric_cols].corr()
    plt.figure(figsize=(10,8))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Matriks Korelasi - Aotizhongxin")
    plt.tight_layout()
    plt.show()

# 5. Distribusi Data PM2.5 untuk stasiun Aotizhongxin
if "Aotizhongxin" in cleaned_dataframes:
    plt.figure(figsize=(10,6))
    sns.histplot(df_aotizhongxin['PM2.5_imputed'], bins=30, kde=True, color="green")
    plt.xlabel("PM2.5 (Imputed)")
    plt.title("Distribusi PM2.5 (Imputed) - Aotizhongxin")
    plt.tight_layout()
    plt.show()

**Insight:**
- Time series plot menunjukkan fluktuasi konsentrasi PM2.5 harian dan jangka panjang
- Terdapat pola musiman yang terlihat dari agregasi bulanan, dengan bulan-bulan tertentu yang memiliki rata-rata PM2.5 lebih tinggi
- Boxplot menunjukkan variasi nilai median dan rentang PM2.5 di berbagai stasiun. Perbedaan ini kemungkinan disebabkan oleh faktor lokal (aktivitas industri, volume lalu lintas, kondisi geografis)
- Proses imputasi berhasil mengurangi outlier ekstrem, terlihat dari penurunan nilai mean sedangkan median relatif stabil
- Distribusi data menjadi lebih representatif setelah cleaning
- Matriks korelasi mengungkap hubungan antara PM2.5 dengan parameter lain (PM10, NO2, CO, O3, dan parameter meteorologi)
- Korelasi tinggi dengan parameter tertentu mengindikasikan kemungkinan sumber polusi yang serupa
- Analisis histogram dan density plot memberikan gambaran sebaran data PM2.5 dan kemungkinan skewness yang dapat mempengaruhi analisis statistik lanjutan

## Visualization & Explanatory Analysis

### Pertanyaan 1:

In [ ]:
df_aotizhongxin = cleaned_dataframes["Aotizhongxin"]

# Time Series Plot
plt.figure(figsize=(12,6))
plt.plot(df_aotizhongxin['datetime'], df_aotizhongxin['PM2.5_imputed'], label="PM2.5 (Imputed)", color="blue")
plt.xlabel("Datetime")
plt.ylabel("PM2.5")
plt.title("Tren Konsentrasi PM2.5 - Aotizhongxin")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

df_aotizhongxin['month'] = df_aotizhongxin['datetime'].dt.month
monthly_avg = df_aotizhongxin.groupby('month')['PM2.5_imputed'].mean()
plt.figure(figsize=(10,6))
monthly_avg.plot(kind='bar', color='skyblue')
plt.xlabel("Bulan")
plt.ylabel("Rata-rata PM2.5")
plt.title("Rata-rata PM2.5 Bulanan - Aotizhongxin")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


1. Tren Konsentrasi PM2.5 dari Waktu ke Waktu (Per Stasiun)
- Time Series Plot:
  Dengan mengagregasi data berdasarkan bulan atau tahun, kita dapat melihat rata-rata PM2.5 dalam periode yang lebih panjang. Bar chart atau line plot agregat (misalnya rata-rata bulanan) akan menyoroti pola musiman dan tren jangka panjang.
  Interpretasi: Jika rata-rata PM2.5 lebih tinggi pada bulan tertentu, hal itu bisa mengindikasikan pengaruh musiman (misalnya kondisi cuaca, pembakaran musiman, dll.).
- Agregasi Bulanan atau Tahunan: Gabungkan data PM2.5 (yang telah diimputasi) dari semua stasiun ke dalam satu DataFrame, lalu buat boxplot untuk masing-masing stasiun.
Interpretasi: Boxplot akan memperlihatkan perbedaan distribusi, nilai median, dan rentang antar stasiun. Jika terdapat stasiun yang consistently menunjukkan nilai yang lebih tinggi, hal ini mengindikasikan perbedaan kualitas udara antar lokasi.


**Insight**
- Dari grafik time series, kita bisa melihat kapan terjadi lonjakan atau penurunan nilai PM2.5.
- Pola agregasi bulanan dapat mengungkapkan musim dengan konsentrasi polusi tertinggi dan memberikan dasar untuk perencanaan kebijakan (misalnya, penerapan pembatasan lalu lintas pada musim dengan polusi tinggi).

### Pertanyaan 2:

In [ ]:
all_data = []
for station, df in cleaned_dataframes.items():
    temp_df = df[['datetime', 'PM2.5_imputed']].copy()
    temp_df['Station'] = station
    all_data.append(temp_df)
all_data_df = pd.concat(all_data)

plt.figure(figsize=(12,6))
sns.boxplot(x='Station', y='PM2.5_imputed', data=all_data_df)
plt.xlabel("Stasiun")
plt.ylabel("PM2.5 (Imputed)")
plt.title("Perbandingan Distribusi PM2.5 Antar Stasiun")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

if "Aotizhongxin" in cleaned_dataframes:
    df_aotizhongxin = cleaned_dataframes["Aotizhongxin"]
    numeric_cols = ["PM2.5_imputed", "PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "RAIN", "WSPM"]
    corr_matrix = df_aotizhongxin[numeric_cols].corr()
    plt.figure(figsize=(10,8))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Matriks Korelasi - Aotizhongxin")
    plt.tight_layout()
    plt.show()


2. Perbedaan Signifikan dalam Kualitas Udara Antar Stasiun dan Faktor Penyebabnya
- Boxplot Antar Stasiun:
Gabungkan data PM2.5 (yang telah diimputasi) dari semua stasiun ke dalam satu DataFrame, lalu buat boxplot untuk masing-masing stasiun.
Interpretasi: Boxplot akan memperlihatkan perbedaan distribusi, nilai median, dan rentang antar stasiun. Jika terdapat stasiun yang consistently menunjukkan nilai yang lebih tinggi, hal ini mengindikasikan perbedaan kualitas udara antar lokasi.
- Heatmap Korelasi Parameter:
Buat matriks korelasi untuk parameter-parameter yang relevan (misalnya, PM2.5, PM10, NO2, CO, O3, serta parameter meteorologi seperti suhu, tekanan, dan kecepatan angin) untuk satu atau beberapa stasiun.
Interpretasi: Hubungan antar parameter dapat mengindikasikan faktor penyebab. Misalnya, korelasi yang tinggi antara PM2.5 dan PM10 bisa mengindikasikan sumber polusi yang sama. Begitu pula dengan korelasi antara PM2.5 dan parameter meteorologi yang bisa menunjukkan kondisi cuaca yang mempengaruhi penyebaran polutan.
- Violin Plot atau Bar Chart Rata-Rata:
Selain boxplot, kita juga dapat membuat grafik rata-rata atau violin plot untuk melihat distribusi dan variabilitas nilai PM2.5 antar stasiun.

**Insight**
- Perbedaan Kualitas Udara:
Boxplot menunjukkan apakah stasiun tertentu (misalnya, stasiun dengan median PM2.5 yang lebih tinggi) memiliki kualitas udara yang lebih buruk dibandingkan stasiun lainnya.
- Potensi Faktor Penyebab:
Heatmap korelasi memberikan gambaran tentang bagaimana PM2.5 berhubungan dengan parameter lain. Jika PM2.5 berkorelasi tinggi dengan PM10, NO2, atau CO, mungkin sumber polusi yang serupa (misalnya, kendaraan bermotor atau aktivitas industri) berperan besar. Korelasi dengan parameter meteorologi juga dapat mengungkap apakah kondisi cuaca tertentu berpengaruh terhadap penyebaran polutan.

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2